In [1]:
import pickle 
import numpy as np 
import pandas as pd
import networkx as nx
from tqdm import tqdm
import time
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
import community
import collections

from src.utils.data_utils import *
from src.utils.general_utils import *
from src.utils.mapping import *
from src.utils.langue import *

In [2]:
movie_path = 'data/Movie.pkl'
actors_path = 'data/Actor.pkl'

with open(movie_path, 'rb') as file:
    Movie = pickle.load(file)
Movie = Movie.reset_index()
with open(actors_path, 'rb') as file:
    Actor = pickle.load(file)

C:\Users\Thibault Schiesser\AppData\Local\Temp\ipykernel_23900\2668311966.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  Movie = pickle.load(file)


In [3]:
nombre_entrees = Movie.Movie_languages.str.contains(',', na=False).sum()

In [4]:
nombre_entrees

np.int64(7888)

In [5]:
Movie.Movie_languages.shape

(61042,)

In [6]:
a = create_actor_language_dataset(Movie, Actor, 5)

We lose 13.58% of the dataset of movies with this operation.


In [7]:
presence = create_cross_language(a)
compte = create_cross_language_count(a)

In [8]:
presence['English Language']

,/m/0bxl4_7,/m/0bxl4__,/m/0bxl4_h,/m/0bxl4_r,/m/0bxl4z5,/m/0bxl4z_,/m/0bxl4zs,/m/0bxl506,/m/0k6ks2,/m/0k6ks8,...,/m/05qrj_,/m/01f46l,/m/0k19q1b,/m/02sty,/m/04jhqtq,/m/05b349c,/m/0bzh52,/m/0n28yxr,/m/01m54f8,sum
Movie_languages,,,,,,,,,,,,,,,,,,,,,
English Language,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,61992.0
Hindi Language,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177.0
Spanish Language,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,426.0
Italian Language,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,734.0
French Language,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,749.0


In [20]:
c = compte['English Language'][compte['English Language'].nb_movie>5]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [17]:
c["pourcentage_non_anglais"] = c.iloc[:,1:4].sum(axis=1)/c["nb_movie"]

C:\Users\Thibault Schiesser\AppData\Local\Temp\ipykernel_23900\190803174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c["pourcentage_non_anglais"] = c.iloc[:,1:4].sum(axis=1)/c["nb_movie"]


In [18]:
c[c["pourcentage_non_anglais"]>0]

Movie_languages,English Language,Hindi Language,Spanish Language,Italian Language,French Language,nb_movie,pourcentage_non_anglais
/m/011xjd,60.0,0.0,1.0,0.0,0.0,61.0,0.016393
/m/043gj,155.0,0.0,0.0,1.0,0.0,156.0,0.006410
/m/02qj31,21.0,0.0,13.0,0.0,0.0,34.0,0.382353
/m/017g2y,65.0,0.0,0.0,1.0,0.0,66.0,0.015152
/m/0hpz8,73.0,0.0,0.0,1.0,0.0,74.0,0.013514
...,...,...,...,...,...,...,...
/m/026t8jg,1.0,0.0,5.0,1.0,0.0,7.0,0.857143
/m/026ksyr,1.0,8.0,0.0,0.0,0.0,9.0,0.888889
/m/04y0hj,1.0,27.0,0.0,0.0,0.0,28.0,0.964286
/m/0b781h6,1.0,10.0,0.0,0.0,0.0,11.0,0.909091


In [19]:
Actor

,Freebase_actor_ID,actor_name,actor_DOB,actor_gender,actor_height,ethnicity,Freebase_movie_ID,actor_age_atmovierelease
0,/m/010p3,Adam Carolla,1964-05-27,M,1.88,Italian Americans,"[/m/0bq2wj, /m/08hjr9, /m/06zm9p7, /m/06c5z_, ...","[35.0, 35.0, 45.0, 33.0, 33.0, 34.0, 42.0, 35...."
1,/m/010q36,Fred Rogers,1928-03-20,M,NaN,None,[/m/033pf1],[67.0]
2,/m/010wx,Aria Giovanni,1977-11-03,F,1.68,Yugoslavs,[/m/07gx0c],[25.0]
3,/m/010xjr,Richard Harris,1930-10-01,M,1.85,Irish people,"[/m/0c_6bf, /m/078mm1, /m/03qfyt, /m/0cz7_4w, ...","[66.0, 46.0, <NA>, 59.0, 48.0, 39.0, 66.0, 50...."
4,/m/0112yl,Toshiro Mifune,1920-04-01,M,1.74,None,"[/m/0513nd, /m/05c2g3q, /m/04wjg1, /m/01tch8, ...","[31.0, 42.0, 45.0, 41.0, 62.0, 29.0, 30.0, 38...."
...,...,...,...,...,...,...,...,...
135755,/m/0z58,Ardal O'Hanlon,1965-10-08,M,NaN,None,[/m/027cvd8],[<NA>]
135756,/m/0z9q0,"José Luis Rodríguez ""El Puma""",1943-01-14,M,NaN,None,[/m/0gls5qd],[69.0]
135757,/m/0zcb7,Marcus Giamatti,1961-10-03,M,1.85,None,"[/m/080kkcx, /m/09v460, /m/02drs7, /m/0gls5v_,...","[38.0, 29.0, 40.0, 48.0, 29.0, 46.0]"
135758,/m/0zcbl,Paul Giamatti,1967-06-06,M,1.74,Irish Americans,"[/m/0ck5rr, /m/04yg13l, /m/04wdfw, /m/03whlgj,...","[38.0, 43.0, 28.0, 41.0, 46.0, 43.0, 28.0, 31...."


regardons si nous avons la même proportion de language de langue pour chaque nb_movie <br> on considère 4 groupes :
- 1 <= nb_film <= 2
- 3 <= nb_film <= 5
- 5 < nb_movie <= 10
- nb_movie > 10
 
<br> h0 : la proportion des langues est la même dans chaque groupe

In [25]:
a["nb_movie"]= a.sum(axis=1)

In [51]:
dict_group = {}
group1 = a[a["nb_movie"]<3]
dict_group[1] = group1
group2 = a[(a["nb_movie"]>2) & (a["nb_movie"]<6)]
dict_group[2] = group2
group3 = a[(a["nb_movie"]<11) & (a["nb_movie"]>5)]
dict_group[3] = group3
group4 = a[a["nb_movie"]>10]
dict_group[4] = group4
print(group1.shape[0] + group2.shape[0] + group3.shape[0] + group4.shape[0] == a.shape[0])

True


In [78]:
nb_group = 4
dict_total = {}

for g in range(1, nb_group + 1):
    group = dict_group[g]
    total_movie = group.nb_movie.sum()
    
    for i in group.columns:
        if i != "nb_movie":
            ratio = float(group[i].sum() / total_movie)
    
            if i in dict_total:
                dict_total[i].append(ratio)
            else:
                dict_total[i] = [ratio]

print(dict_total)

{'English Language': [0.7979323866674588, 0.8204991087344029, 0.8393774903274239, 0.8215969535783365], 'Hindi Language': [0.028073198265106056, 0.042572786690433746, 0.06406998902812265, 0.1378949065119278], 'Spanish Language': [0.060543045570673165, 0.0421865715983363, 0.021568400993243635, 0.007464941972920696], 'Italian Language': [0.04310498484938506, 0.03933452168746286, 0.03049026967719582, 0.01606826241134752], 'French Language': [0.07034638464737686, 0.05540701128936423, 0.044493849974013976, 0.01697493552546744]}


In [69]:
from scipy.stats import chi2_contingency

In [79]:
df = pd.DataFrame(dict_total, index=[1,2,3,4])

In [88]:
chi2, p_value, dof, expected = chi2_contingency(df)

In [89]:
# Résultats
print("Statistique du Chi-2 :", chi2)
print("P-value :", p_value)
print("Degrés de liberté :", dof)
print("Tableau attendu théorique :\n", expected)

# Conclusion
if p_value < 0.05:
    print("Les distributions sont statistiquement différentes (p < 0.05).")
else:
    print("Aucune différence significative entre les distributions (p >= 0.05).")

Statistique du Chi-2 : 0.20116474096698686
P-value : 0.9999999986805574
Degrés de liberté : 12
Tableau attendu théorique :
 [[0.81985148 0.06815272 0.03294074 0.03224951 0.04680555]
 [0.81985148 0.06815272 0.03294074 0.03224951 0.04680555]
 [0.81985148 0.06815272 0.03294074 0.03224951 0.04680555]
 [0.81985148 0.06815272 0.03294074 0.03224951 0.04680555]]
Aucune différence significative entre les distributions (p >= 0.05).


In [83]:
chi2

np.float64(0.20116474096698686)

In [84]:
p_value

np.float64(0.9999999986805574)

In [85]:
dof

12